In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset2 = pd.read_csv('../dataset2/dataset2_before_splitting.csv')

# TRAITEMENT SUR LE DATASET 2 (INTERPOLATED)

## Split en train/test

In [3]:
## On repasse le type des colonnes 'was_interpolated' en int au lieu de bool

cols_to_convert = [
    c for c in dataset2.columns 
    if "_was_interpolated" in c
]

for col in cols_to_convert:
    dataset2[col] = dataset2[col].astype(int)

In [4]:
#dataset2 = dataset2.drop(columns=['Unnamed: 0'])
#dataset2.to_csv('dalas/project/githubProject/ConflictPrediction/dataset2/dataset2_before_splitting.csv', index=False)

In [5]:
## Code pour split le dataset 2 en train/test 90-10 %

X = dataset2.drop(columns=['war'])
y = dataset2['war']

# Split train/test final
X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.1,# 10% pour test
    stratify=y, #pour garder la répartition des classes
    random_state=42 #pour mettre une random seed fixe
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (6610, 166)
Test shape: (735, 166)


## Entraînement MLP avec StratifiedKFold + scores

In [16]:
def pipeline_stratifiedKFold_and_training_MLP(X_train, y_train, k=5):
    
    list_accuracy_scores = []
    list_precision_scores = []
    list_recall_scores = []
    list_f1_scores = []

    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]

        print(f"-- Fold {i} --")

        # Modèle MLP
        model = MLPClassifier(
            hidden_layer_sizes=(128, 64),
            activation='relu',
            solver='adam',
            max_iter=200,
            random_state=42
        )

        # Entraînement
        model.fit(X_tr, y_tr)

        # Prédiction
        y_pred = model.predict(X_val)

        # Métriques
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred)
        rec = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)

        list_accuracy_scores.append(acc)
        list_precision_scores.append(prec)
        list_recall_scores.append(rec)
        list_f1_scores.append(f1)

        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    # Moyenne et écart-type pour chaque métrique
    avg_acc, std_acc = np.mean(list_accuracy_scores), np.std(list_accuracy_scores)
    avg_prec, std_prec = np.mean(list_precision_scores), np.std(list_precision_scores)
    avg_rec, std_rec = np.mean(list_recall_scores), np.std(list_recall_scores)
    avg_f1, std_f1 = np.mean(list_f1_scores), np.std(list_f1_scores)

    print("\nAverage metrics over {} folds:".format(k))
    print(f"Accuracy  : {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision : {avg_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall    : {avg_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score  : {avg_f1:.4f} ± {std_f1:.4f}")

    return ((avg_acc, std_acc), (avg_prec, std_prec), (avg_rec, std_rec), (avg_f1, std_f1))

((avg_acc_mlp, std_acc_mlp),
 (avg_prec_mlp, std_prec_mlp),
 (avg_rec_mlp, std_rec_mlp),
 (avg_f1_mlp, std_f1_mlp)) = pipeline_stratifiedKFold_and_training_MLP(X_train, y_train, k=5)


-- Fold 0 --
Accuracy : 0.8903
Precision: 0.8660
Recall   : 0.8612
F1-score : 0.8636
-- Fold 1 --
Accuracy : 0.9017
Precision: 0.8711
Recall   : 0.8874
F1-score : 0.8792
-- Fold 2 --
Accuracy : 0.9274
Precision: 0.9024
Recall   : 0.9193
F1-score : 0.9108
-- Fold 3 --
Accuracy : 0.9032
Precision: 0.8872
Recall   : 0.8705
F1-score : 0.8788
-- Fold 4 --
Accuracy : 0.8464
Precision: 0.8300
Recall   : 0.7786
F1-score : 0.8035

Average metrics over 5 folds:
Accuracy  : 0.8938 ± 0.0266
Precision : 0.8713 ± 0.0243
Recall    : 0.8634 ± 0.0468
F1-score  : 0.8672 ± 0.0354


## Entraînement XGBoost avec StratifiedKFold + scores

### SKF avec K =10 (on garde le même skf pour avoir la meme répartition)

In [17]:
def pipeline_stratifiedKFold_and_training_XGBoost(X_train, y_train, k=10):
    
    list_accuracy_scores = []
    list_precision_scores = []
    list_recall_scores = []
    list_f1_scores = []

    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42) # On garde le même skf pour la comparaison

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]

        print(f"-- Fold {i} --")

        # Modèle MLP
        model = xgb.XGBClassifier( # Hyperparamètres à revoir
            n_estimators=100,
            max_depth=5,
            learning_rate=1,
            random_state=42
        )

        # Entraînement
        model.fit(X_tr, y_tr)

        # Prédiction
        y_pred = model.predict(X_val)

        # Calcul des métriques
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred)
        rec = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)

        list_accuracy_scores.append(acc)
        list_precision_scores.append(prec)
        list_recall_scores.append(rec)
        list_f1_scores.append(f1)

        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    # Moyenne et écart-type pour chaque métrique
    avg_acc, std_acc = np.mean(list_accuracy_scores), np.std(list_accuracy_scores)
    avg_prec, std_prec = np.mean(list_precision_scores), np.std(list_precision_scores)
    avg_rec, std_rec = np.mean(list_recall_scores), np.std(list_recall_scores)
    avg_f1, std_f1 = np.mean(list_f1_scores), np.std(list_f1_scores)

    print("\nAverage metrics over {} folds:".format(k))
    print(f"Accuracy  : {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision : {avg_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall    : {avg_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score  : {avg_f1:.4f} ± {std_f1:.4f}")

    return ((avg_acc, std_acc), (avg_prec, std_prec), (avg_rec, std_rec), (avg_f1, std_f1))

# Exécution sur le dataset 2
((avg_acc_xgb, std_acc_xgb),
 (avg_prec_xgb, std_prec_xgb),
 (avg_rec_xgb, std_rec_xgb),
 (avg_f1_xgb, std_f1_xgb)) = pipeline_stratifiedKFold_and_training_XGBoost(X_train, y_train, k=10)

-- Fold 0 --
Accuracy : 0.9470
Precision: 0.9462
Recall   : 0.9213
F1-score : 0.9336
-- Fold 1 --
Accuracy : 0.9455
Precision: 0.9326
Recall   : 0.9326
F1-score : 0.9326
-- Fold 2 --
Accuracy : 0.9486
Precision: 0.9464
Recall   : 0.9251
F1-score : 0.9356
-- Fold 3 --
Accuracy : 0.9531
Precision: 0.9538
Recall   : 0.9288
F1-score : 0.9412
-- Fold 4 --
Accuracy : 0.9516
Precision: 0.9608
Recall   : 0.9176
F1-score : 0.9387
-- Fold 5 --
Accuracy : 0.9546
Precision: 0.9504
Recall   : 0.9361
F1-score : 0.9432
-- Fold 6 --
Accuracy : 0.9531
Precision: 0.9273
Recall   : 0.9586
F1-score : 0.9427
-- Fold 7 --
Accuracy : 0.9516
Precision: 0.9432
Recall   : 0.9361
F1-score : 0.9396
-- Fold 8 --
Accuracy : 0.9259
Precision: 0.9094
Recall   : 0.9060
F1-score : 0.9077
-- Fold 9 --
Accuracy : 0.9576
Precision: 0.9612
Recall   : 0.9323
F1-score : 0.9466

Average metrics over 10 folds:
Accuracy  : 0.9489 ± 0.0084
Precision : 0.9431 ± 0.0152
Recall    : 0.9295 ± 0.0132
F1-score  : 0.9361 ± 0.0104


## Entraînement CatBoost avec StratifiedKFold + scores

In [10]:
def pipeline_stratifiedKFold_and_training_CatBoost(X_train, y_train, k=10):

    list_accuracy_scores = []
    list_precision_scores = []
    list_recall_scores = []
    list_f1_scores = []
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):

        # Récupération des sous-ensembles
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]

        print(f"Fold {i}")

        # Modèle CatBoost
        model = CatBoostClassifier(
            iterations=300,
            learning_rate=0.05,
            depth=6,
            loss_function="Logloss",
            eval_metric="F1",
            random_seed=42,
            verbose=False
        )

        # Entraînement
        model.fit(X_tr, y_tr)

        # Prédiction
        y_pred = model.predict(X_val)

        # Métriques
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred)
        rec = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)

        list_accuracy_scores.append(acc)
        list_precision_scores.append(prec)
        list_recall_scores.append(rec)
        list_f1_scores.append(f1)

        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    # Moyenne et écart-type pour chaque métrique
    avg_acc, std_acc = np.mean(list_accuracy_scores), np.std(list_accuracy_scores)
    avg_prec, std_prec = np.mean(list_precision_scores), np.std(list_precision_scores)
    avg_rec, std_rec = np.mean(list_recall_scores), np.std(list_recall_scores)
    avg_f1, std_f1 = np.mean(list_f1_scores), np.std(list_f1_scores)

    print("\nAverage metrics over {} folds:".format(k))
    print(f"Accuracy  : {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision : {avg_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall    : {avg_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score  : {avg_f1:.4f} ± {std_f1:.4f}")

    return ((avg_acc, std_acc), (avg_prec, std_prec), (avg_rec, std_rec), (avg_f1, std_f1))

((avg_acc_cb, std_acc_cb),
 (avg_prec_cb, std_prec_cb),
 (avg_rec_cb, std_rec_cb), 
 (avg_f1_cb, std_f1_cb)) = pipeline_stratifiedKFold_and_training_CatBoost(X_train, y_train, k=10)

Fold 0
Accuracy : 0.9077
Precision: 0.9055
Recall   : 0.8614
F1-score : 0.8829
Fold 1
Accuracy : 0.9304
Precision: 0.9266
Recall   : 0.8989
F1-score : 0.9125
Fold 2
Accuracy : 0.9395
Precision: 0.9486
Recall   : 0.8989
F1-score : 0.9231
Fold 3
Accuracy : 0.9289
Precision: 0.9435
Recall   : 0.8764
F1-score : 0.9087
Fold 4
Accuracy : 0.9259
Precision: 0.9325
Recall   : 0.8801
F1-score : 0.9056
Fold 5
Accuracy : 0.9259
Precision: 0.9323
Recall   : 0.8797
F1-score : 0.9052
Fold 6
Accuracy : 0.9274
Precision: 0.9431
Recall   : 0.8722
F1-score : 0.9062
Fold 7
Accuracy : 0.9259
Precision: 0.9357
Recall   : 0.8759
F1-score : 0.9049
Fold 8
Accuracy : 0.9017
Precision: 0.9205
Recall   : 0.8271
F1-score : 0.8713
Fold 9
Accuracy : 0.9183
Precision: 0.9344
Recall   : 0.8571
F1-score : 0.8941

Average metrics over 10 folds:
Accuracy  : 0.9231 ± 0.0106
Precision : 0.9323 ± 0.0119
Recall    : 0.8728 ± 0.0199
F1-score  : 0.9015 ± 0.0142


## Entraînement GradientBoosting avec StratifiedKFold + scores

In [12]:
def pipeline_stratifiedKFold_and_training_GB(X_train, y_train, k=10):

    list_accuracy_scores = []
    list_precision_scores = []
    list_recall_scores = []
    list_f1_scores = []

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]

        print(f"-- Fold {i} --")

        # Modèle Gradient Boosting
        modelGB = GradientBoostingClassifier(
            n_estimators=200,
            learning_rate=0.05,
            max_depth=3
        )

        # Entraînement
        modelGB.fit(X_tr, y_tr)

        # Prédiction
        y_predGB = modelGB.predict(X_val)

        # Calcul des métriques
        acc = accuracy_score(y_val, y_predGB)
        prec = precision_score(y_val, y_predGB)
        rec = recall_score(y_val, y_predGB)
        f1 = f1_score(y_val, y_predGB)

        list_accuracy_scores.append(acc)
        list_precision_scores.append(prec)
        list_recall_scores.append(rec)
        list_f1_scores.append(f1)

        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    # Moyenne et écart-type
    avg_acc, std_acc = np.mean(list_accuracy_scores), np.std(list_accuracy_scores)
    avg_prec, std_prec = np.mean(list_precision_scores), np.std(list_precision_scores)
    avg_rec, std_rec = np.mean(list_recall_scores), np.std(list_recall_scores)
    avg_f1, std_f1 = np.mean(list_f1_scores), np.std(list_f1_scores)

    print("\nAverage metrics over {} folds:".format(k))
    print(f"Accuracy  : {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision : {avg_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall    : {avg_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score  : {avg_f1:.4f} ± {std_f1:.4f}")

    return ((avg_acc, std_acc), (avg_prec, std_prec), (avg_rec, std_rec), (avg_f1, std_f1))

# Exécution
((avg_acc_gb, std_acc_gb),
 (avg_prec_gb, std_prec_gb),
 (avg_rec_gb, std_rec_gb),
 (avg_f1_gb, std_f1_gb)) = pipeline_stratifiedKFold_and_training_GB(X_train, y_train, k=10)


-- Fold 0 --
Accuracy : 0.7806
Precision: 0.8020
Recall   : 0.6067
F1-score : 0.6908
-- Fold 1 --
Accuracy : 0.7791
Precision: 0.7951
Recall   : 0.6105
F1-score : 0.6907
-- Fold 2 --
Accuracy : 0.7685
Precision: 0.8032
Recall   : 0.5655
F1-score : 0.6637
-- Fold 3 --
Accuracy : 0.7897
Precision: 0.8636
Recall   : 0.5693
F1-score : 0.6862
-- Fold 4 --
Accuracy : 0.7973
Precision: 0.8595
Recall   : 0.5955
F1-score : 0.7035
-- Fold 5 --
Accuracy : 0.7716
Precision: 0.8286
Recall   : 0.5451
F1-score : 0.6576
-- Fold 6 --
Accuracy : 0.7700
Precision: 0.8032
Recall   : 0.5677
F1-score : 0.6652
-- Fold 7 --
Accuracy : 0.7821
Precision: 0.8547
Recall   : 0.5526
F1-score : 0.6712
-- Fold 8 --
Accuracy : 0.7670
Precision: 0.7917
Recall   : 0.5714
F1-score : 0.6638
-- Fold 9 --
Accuracy : 0.7927
Precision: 0.8486
Recall   : 0.5902
F1-score : 0.6962

Average metrics over 10 folds:
Accuracy  : 0.7799 ± 0.0102
Precision : 0.8250 ± 0.0276
Recall    : 0.5775 ± 0.0211
F1-score  : 0.6789 ± 0.0155


## Entraînement ExtraTrees avec StratifiedKFold + scores

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def pipeline_stratifiedKFold_and_training_ET(X_train, y_train, k=10):

    list_accuracy_scores = []
    list_precision_scores = []
    list_recall_scores = []
    list_f1_scores = []

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]

        print(f"-- Fold {i} --")

        # Modèle ExtraTrees
        modelET = ExtraTreesClassifier(
            n_estimators=300,
            max_depth=None,
            random_state=42,
            n_jobs=-1
        )

        # Entraînement
        modelET.fit(X_tr, y_tr)

        # Prédiction
        y_predET = modelET.predict(X_val)

        # Métriques
        acc = accuracy_score(y_val, y_predET)
        prec = precision_score(y_val, y_predET)
        rec = recall_score(y_val, y_predET)
        f1 = f1_score(y_val, y_predET)

        list_accuracy_scores.append(acc)
        list_precision_scores.append(prec)
        list_recall_scores.append(rec)
        list_f1_scores.append(f1)

        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    # Moyenne et écart-type
    avg_acc, std_acc = np.mean(list_accuracy_scores), np.std(list_accuracy_scores)
    avg_prec, std_prec = np.mean(list_precision_scores), np.std(list_precision_scores)
    avg_rec, std_rec = np.mean(list_recall_scores), np.std(list_recall_scores)
    avg_f1, std_f1 = np.mean(list_f1_scores), np.std(list_f1_scores)

    print("\nAverage metrics over {} folds:".format(k))
    print(f"Accuracy  : {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision : {avg_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall    : {avg_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score  : {avg_f1:.4f} ± {std_f1:.4f}")

    return ((avg_acc, std_acc), (avg_prec, std_prec), (avg_rec, std_rec), (avg_f1, std_f1))

# Exécution
((avg_acc_et, std_acc_et),
 (avg_prec_et, std_prec_et),
 (avg_rec_et, std_rec_et),
 (avg_f1_et, std_f1_et)) = pipeline_stratifiedKFold_and_training_ET(X_train, y_train, k=10)


-- Fold 0 --
Accuracy : 0.9728
Precision: 0.9698
Recall   : 0.9625
F1-score : 0.9662
-- Fold 1 --
Accuracy : 0.9607
Precision: 0.9582
Recall   : 0.9438
F1-score : 0.9509
-- Fold 2 --
Accuracy : 0.9773
Precision: 0.9737
Recall   : 0.9700
F1-score : 0.9719
-- Fold 3 --
Accuracy : 0.9607
Precision: 0.9513
Recall   : 0.9513
F1-score : 0.9513
-- Fold 4 --
Accuracy : 0.9713
Precision: 0.9627
Recall   : 0.9663
F1-score : 0.9645
-- Fold 5 --
Accuracy : 0.9758
Precision: 0.9845
Recall   : 0.9549
F1-score : 0.9695
-- Fold 6 --
Accuracy : 0.9743
Precision: 0.9734
Recall   : 0.9624
F1-score : 0.9679
-- Fold 7 --
Accuracy : 0.9697
Precision: 0.9695
Recall   : 0.9549
F1-score : 0.9621
-- Fold 8 --
Accuracy : 0.9546
Precision: 0.9504
Recall   : 0.9361
F1-score : 0.9432
-- Fold 9 --
Accuracy : 0.9682
Precision: 0.9767
Recall   : 0.9436
F1-score : 0.9598

Average metrics over 10 folds:
Accuracy  : 0.9685 ± 0.0071
Precision : 0.9670 ± 0.0106
Recall    : 0.9546 ± 0.0104
F1-score  : 0.9607 ± 0.0089


## Entraînement RandomForest avec StratifiedKFold + scores

In [15]:
def pipeline_stratifiedKFold_and_training_RF(X_train, y_train, k=10):

    list_accuracy_scores = []
    list_precision_scores = []
    list_recall_scores = []
    list_f1_scores = []
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):

        # Récupération des sous-ensembles
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[val_index]

        print(f"Fold {i}")

        # Modèle RandomForest
        model = RandomForestClassifier(
            n_estimators=300, # en ligne avec LGBM/CatBoost
            max_depth=None, # laisse l’arbre croître (fort sur F1)
            min_samples_split=2,
            min_samples_leaf=1,
            class_weight="balanced", # Essentiel pour le dataset 60/40
            random_state=42,
            n_jobs=-1
        )

        # Entraînement
        model.fit(X_tr, y_tr)

        # Prédiction
        y_pred = model.predict(X_val)

        # Métriques
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred)
        rec = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)

        list_accuracy_scores.append(acc)
        list_precision_scores.append(prec)
        list_recall_scores.append(rec)
        list_f1_scores.append(f1)

        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")

    # Moyenne et écart-type pour chaque métrique
    avg_acc, std_acc = np.mean(list_accuracy_scores), np.std(list_accuracy_scores)
    avg_prec, std_prec = np.mean(list_precision_scores), np.std(list_precision_scores)
    avg_rec, std_rec = np.mean(list_recall_scores), np.std(list_recall_scores)
    avg_f1, std_f1 = np.mean(list_f1_scores), np.std(list_f1_scores)

    print("\nAverage metrics over {} folds:".format(k))
    print(f"Accuracy  : {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision : {avg_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall    : {avg_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score  : {avg_f1:.4f} ± {std_f1:.4f}")

    return ((avg_acc, std_acc), (avg_prec, std_prec), (avg_rec, std_rec), (avg_f1, std_f1))

((avg_acc_rf, std_acc_rf),
 (avg_prec_rf, std_prec_rf),
 (avg_rec_rf, std_rec_rf),
 (avg_f1_rf, std_f1_rf)) = pipeline_stratifiedKFold_and_training_RF(X_train, y_train, k=10)

Fold 0
Accuracy : 0.9516
Precision: 0.9572
Recall   : 0.9213
F1-score : 0.9389
Fold 1
Accuracy : 0.9516
Precision: 0.9537
Recall   : 0.9251
F1-score : 0.9392
Fold 2
Accuracy : 0.9622
Precision: 0.9549
Recall   : 0.9513
F1-score : 0.9531
Fold 3
Accuracy : 0.9592
Precision: 0.9688
Recall   : 0.9288
F1-score : 0.9484
Fold 4
Accuracy : 0.9561
Precision: 0.9648
Recall   : 0.9251
F1-score : 0.9446
Fold 5
Accuracy : 0.9652
Precision: 0.9841
Recall   : 0.9286
F1-score : 0.9555
Fold 6
Accuracy : 0.9713
Precision: 0.9805
Recall   : 0.9474
F1-score : 0.9637
Fold 7
Accuracy : 0.9561
Precision: 0.9611
Recall   : 0.9286
F1-score : 0.9446
Fold 8
Accuracy : 0.9410
Precision: 0.9558
Recall   : 0.8947
F1-score : 0.9243
Fold 9
Accuracy : 0.9546
Precision: 0.9646
Recall   : 0.9211
F1-score : 0.9423

Average metrics over 10 folds:
Accuracy  : 0.9569 ± 0.0079
Precision : 0.9645 ± 0.0100
Recall    : 0.9272 ± 0.0146
F1-score  : 0.9454 ± 0.0102


## Tableau recapitulatif

In [19]:
import pandas as pd

data = {
    'Model': ['MLP', 'XGBoost', 'CatBoost', 'GradientBoosting', 'ExtraTrees', 'RandomForest'],
    'Accuracy (avg ± std)': [
        f"{avg_acc_mlp:.4f} ± {std_acc_mlp:.4f}",
        f"{avg_acc_xgb:.4f} ± {std_acc_xgb:.4f}",
        f"{avg_acc_cb:.4f} ± {std_acc_cb:.4f}",
        f"{avg_acc_gb:.4f} ± {std_acc_gb:.4f}",
        f"{avg_acc_et:.4f} ± {std_acc_et:.4f}",
        f"{avg_acc_rf:.4f} ± {std_acc_rf:.4f}"
    ],
    'Precision (avg ± std)': [
        f"{avg_prec_mlp:.4f} ± {std_prec_mlp:.4f}",
        f"{avg_prec_xgb:.4f} ± {std_prec_xgb:.4f}",
        f"{avg_prec_cb:.4f} ± {std_prec_cb:.4f}",
        f"{avg_prec_gb:.4f} ± {std_prec_gb:.4f}",
        f"{avg_prec_et:.4f} ± {std_prec_et:.4f}",
        f"{avg_prec_rf:.4f} ± {std_prec_rf:.4f}"
    ],
    'Recall (avg ± std)': [
        f"{avg_rec_mlp:.4f} ± {std_rec_mlp:.4f}",
        f"{avg_rec_xgb:.4f} ± {std_rec_xgb:.4f}",
        f"{avg_rec_cb:.4f} ± {std_rec_cb:.4f}",
        f"{avg_rec_gb:.4f} ± {std_rec_gb:.4f}",
        f"{avg_rec_et:.4f} ± {std_rec_et:.4f}",
        f"{avg_rec_rf:.4f} ± {std_rec_rf:.4f}"
    ],
    'F1-score (avg ± std)': [
        f"{avg_f1_mlp:.4f} ± {std_f1_mlp:.4f}",
        f"{avg_f1_xgb:.4f} ± {std_f1_xgb:.4f}",
        f"{avg_f1_cb:.4f} ± {std_f1_cb:.4f}",
        f"{avg_f1_gb:.4f} ± {std_f1_gb:.4f}",
        f"{avg_f1_et:.4f} ± {std_f1_et:.4f}",
        f"{avg_f1_rf:.4f} ± {std_f1_rf:.4f}"
    ]
}

df = pd.DataFrame(data)
df


,Model,Accuracy (avg ± std),Precision (avg ± std),Recall (avg ± std),F1-score (avg ± std)
0,MLP,0.8938 ± 0.0266,0.8713 ± 0.0243,0.8634 ± 0.0468,0.8672 ± 0.0354
1,XGBoost,0.9489 ± 0.0084,0.9431 ± 0.0152,0.9295 ± 0.0132,0.9361 ± 0.0104
2,CatBoost,0.9231 ± 0.0106,0.9323 ± 0.0119,0.8728 ± 0.0199,0.9015 ± 0.0142
3,GradientBoosting,0.7799 ± 0.0102,0.8250 ± 0.0276,0.5775 ± 0.0211,0.6789 ± 0.0155
4,ExtraTrees,0.9685 ± 0.0071,0.9670 ± 0.0106,0.9546 ± 0.0104,0.9607 ± 0.0089
5,RandomForest,0.9569 ± 0.0079,0.9645 ± 0.0100,0.9272 ± 0.0146,0.9454 ± 0.0102
